In [ ]:
# Sys
import sys
# !{sys.executable} --version

# !{sys.executable} -m pip uninstall -y torch torchvision torchaudio
# !mamba uninstall --yes --prefix {sys.prefix} pytorch torchvision torchaudio cudatoolkit=11.3
# !mamba uninstall --yes --prefix {sys.prefix} dgl dgl-cuda11.3


# !mamba install --yes --prefix {sys.prefix} logzero
# !conda uninstall --yes pytorch torchvision torchaudio
# !mamba install --yes --prefix {sys.prefix} -c rapidsai-nightly -c nvidia -c numba -c conda-forge cudf cudatoolkit


# !{sys.executable} -m pip install imag

import seaborn as sns
import numpy as np
import pandas as pd
from time import perf_counter
import os
from glob import glob
import matplotlib.pyplot as plt
import copy
import scipy as sp

# Torch
import torch
print(torch.__version__)
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# Loading bar
from tqdm import trange

# Logger
from logzero import logger


import warnings
warnings.simplefilter("ignore", UserWarning)


from itertools import permutations, combinations,combinations_with_replacement


import re
import pandas as pd
import pickle
#random
from time import perf_counter

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

from tqdm import tqdm

In [ ]:
# !nvidia-smi

In [ ]:
H_path = "../hydrogen_comps/minimal_2e_2o/chains/even/"

In [ ]:
with open(os.path.join(H_path,'test_ind.pickle'), 'rb') as handle:
    test_ind = pickle.load(handle)

with open(os.path.join(H_path,'train_ind.pickle'), 'rb') as handle:
    train_ind = pickle.load(handle)
    
print(len(train_ind),len(test_ind))    


In [ ]:
feat_name=['From_Same_Orbital', 'pair_energy', 'coulomb', 'screen1_1', 'screen1_2',
       'screen1_3', 'screen1_4', 'screen2_1', 'screen2_2', 'screen2_3',
       'screen2_4', 'eijab_1', 'eijab_2', 'eijab_3', 'eijab_4', 'screenvirt_1',
       'screenvirt_2', 'screenvirt_3', 'screenvirt_4', 'Fr1', 'Fr2', 'Fr3',
       'Fr4', 'Fs1', 'Fs2', 'Fs3', 'Fs4', 'occr1', 'occr2', 'occr3', 'occr4',
       'occs1', 'occs2', 'occs3', 'occs4', 'SCFFr1', 'SCFFr2', 'SCFFr3',
       'SCFFr4', 'SCFFs1', 'SCFFs2', 'SCFFs3', 'SCFFs4', 'SCFOCCr1',
       'SCFOCCr2', 'SCFOCCr3', 'SCFOCCr4', 'SCFOCCs1', 'SCFOCCs2', 'SCFOCCs3',
       'SCFOCCs4', 'hrr1', 'hrr2', 'hrr3', 'hrr4', 'hss1', 'hss2', 'hss3',
       'hss4', 'hpp', 'hqq', 'Fp', 'Fq', 'occp', 'occq', 'SCFFp', 'SCFFq',
       'SCFOCCp', 'SCFOCCq']

# feat_name=['occs3',
#  'SCFOCCs2',
#  'SCFOCCs1',
#  'screen2_1',
#  'SCFOCCp',
#  'hss4',
#  'hrr1',
#  'Fs2',
#  'occq',
#  'screenvirt_4',
#  'SCFFs2',
#  'hrr3',
#  'SCFFr1',
#  'screenvirt_2',
#  'screenvirt_3',
#  'SCFFs4',
#  'hss2',
#  'screen1_2',
#  'hrr4',
#  'screen1_3',
#  'Fp',
#  'Fr1',
#  'screenvirt_1',
#  'occs4',
#  'screen2_4',
#  'SCFFr2',
#  'eijab_2',
#  'screen2_3',
#  'SCFFs1',
#  'eijab_4',
#  'Fq',
#  'hss1',
#  'SCFFr4',
#  'occs2',
#  'occp',
#  'screen1_4',
#  'eijab_1',
#  'hpp',
#  'pair_energy',
#  'hrr2',
#  'hqq',
#  'SCFFr3',
#  'occs1',
#  'Fr2',
#  'eijab_3',
#  'SCFOCCq',
#  'SCFOCCs4',
#  'Fs3',
#  'Fs1',
#  'From_Same_Orbital',
#  'hss3',
#  'Fs4',
#  'SCFOCCs3',
#  'Fr4',
#  'screen2_2',
#  'screen1_1',
#  'Fr3',
#  'SCFFs3']

feat_name=['SCFFs3',
 'hrr3',
 'screen1_1',
 'SCFOCCs2',
 'hpp',
 'SCFOCCs3',
 'hss4',
 'screen2_2',
 'SCFOCCq',
 'hss1',
 'screenvirt_3',
 'SCFFs4',
 'SCFFr4',
 'SCFOCCs1',
 'SCFFr3',
 'hss2',
 'screen2_4',
 'screenvirt_2',
 'SCFFs2',
 'hss3',
 'hqq',
 'hrr4',
 'hrr1',
 'screen1_3',
 'SCFOCCs4',
 'SCFFr1',
 'SCFFs1',
 'hrr2',
 'SCFFr2',
 'screen1_4',
 'screen2_1',
 'screenvirt_1',
 'screen1_2',
 'SCFOCCp',
 'screen2_3',
 'screenvirt_4']


In [ ]:
train_X=[]
train_y = []

test_X=[]
test_y = []
# for nam in sorted(glob('*/fixed_feats.pickle'),key=lambda x: int(x.split('_')[0].strip('H')))[2:]:
for nam in glob(os.path.join(H_path,'*/fixed_feats.pickle')):
    trydf=pd.read_pickle(nam).astype(float)[feat_name]    
    Hnam=os.path.basename(os.path.dirname(nam)).split('_')[0]
    for i in train_ind:
        # try:
        #     print("TOP",Hnam,i)
        #     print(trydf.loc[f'{Hnam}_chain/{Hnam}_{i:.2f}'].values.shape)
        #     train_X.append(trydf.loc[f'{Hnam}_chain/{Hnam}_{i:.2f}'].values)
        # except:
        #     print("Bottom",Hnam,i)
        #     print(trydf.loc[f'{Hnam}_chain/{Hnam}_{i:.2f}/'].values.shape)
        train_X.append(trydf.loc[f'{Hnam}_chain/{Hnam}_{i:.2f}/'].values)
    for j in test_ind: 
        # try:
        #     test_X.append(trydf.loc[f'{Hnam}_chain/{Hnam}_{j:.2f}'].values)
        # except:
        test_X.append(trydf.loc[f'{Hnam}_chain/{Hnam}_{j:.2f}/'].values)
    
# This recover is wrong... did not account for the # of pair-energies     
recover_train=[]    
recover_test=[]    
tst_idx=0
tr_idx=0
for nidx,nam in enumerate(glob(os.path.join(H_path,'*/fixed_targets.pickle'))):
    trydf=pd.read_pickle(nam).astype(float)
    Hnam=os.path.basename(os.path.dirname(nam)).split('_')[0]
    for tridx,i in enumerate(train_ind):
        pairdf=trydf[f'{i:.2f}'].values
        train_y.append(pairdf)
        for k in range(len(pairdf)):
            recover_train.append((nam.split('_')[0],tridx,tr_idx))
            tr_idx+=1
        
    for teidx,j in enumerate(test_ind): 
        pairdf=trydf[f'{j:.2f}'].values
        test_y.append(pairdf)        
        
        for k in range(len(pairdf)):
            recover_test.append((nam.split('_')[0],teidx,tst_idx))
            tst_idx+=1


In [ ]:
X_train, X_test=np.vstack(train_X).astype(float),np.vstack(test_X).astype(float)
y_train,y_test=np.hstack(train_y).astype(float),np.hstack(test_y).astype(float)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
for u,v in np.argwhere(np.isnan(X_train)):
    X_train[u,v]=0
    
for u,v in np.argwhere(np.isnan(X_test)):
    X_test[u,v]=0   


scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
# Define custom Dataset
class RegressionDataset(Dataset):
    def __init__(self, features, targets):
        self.X = torch.from_numpy(features).float()
        self.y = torch.from_numpy(targets).float()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



In [ ]:

torch.manual_seed(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)



torch.cuda.empty_cache()
if device.type == 'cuda':
    if torch.cuda.device_count() > 1:
        for i in range(torch.cuda.device_count()):
            print(f'Device {i}')
            print(torch.cuda.get_device_name(i))
            print('Memory Usage:')
            print('Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
            print('Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
    else:
        print(torch.cuda.get_device_name(0))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

torch.backends.cudnn.benchmark = True

In [ ]:
X_train.shape[0] // 32

In [ ]:
796 // 32

In [ ]:
# Create DataLoaders
batch_size = 32 * 24
train_dataset = RegressionDataset(X_train, y_train)
test_dataset = RegressionDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self,featdim,depth=10):
        super().__init__()
        self.inputlayer = torch.nn.Linear(featdim, featdim)
        self.middlelayer =  self.middlelayers = nn.ModuleList([nn.Linear(featdim, featdim) for _ in range(depth)])
        self.outlayer = torch.nn.Linear(featdim, 1)
    def forward(self, x):
        x = F.relu(self.inputlayer(x))
        for layer in self.middlelayers:
            x = F.relu(layer(x))
        return self.outlayer(x)

In [ ]:
np.min(y_train),np.max(y_train),np.mean(y_train),np.median(y_train)

In [ ]:
model = Model(len(feat_name)).to(device)

# Define loss and optimizer
criterion = nn.L1Loss() #nn.MSELoss()
learning_rate = 1e-6
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-3)


# Training loop
epochs = 1000
train_losses = []
test_losses = []
# for epoch in tqdm(range(epochs)):
for epoch in range(epochs):    
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
    # for batch_X, batch_y in tqdm(train_loader, desc="Train"):
        optimizer.zero_grad()
        batch_X=batch_X.to(device)
        batch_y=batch_y.to(device)
        outputs = model(batch_X).flatten()
        loss = criterion(outputs, batch_y)
        # print(outputs.shape, batch_y.shape)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()/batch_X.size(0)
    
    epoch_loss = running_loss #/ len(train_loader.dataset)
    train_losses.append(epoch_loss)
    
    # Evaluation on test set
    model.eval()
    test_running_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
        # for batch_X, batch_y in tqdm(test_loader, desc="Test"):
            batch_X=batch_X.to(device)
            batch_y=batch_y.to(device)            
            outputs = model(batch_X).flatten()
            loss = criterion(outputs, batch_y)
            test_running_loss += loss.item()/batch_X.size(0)
    test_epoch_loss = test_running_loss #/ len(test_loader.dataset)
    test_losses.append(test_epoch_loss)
    
    if (epoch+1) % 100 == 0 or epoch == 0:
        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {epoch_loss:.4e} - Test Loss: {test_epoch_loss:.4e}")





# Evaluation
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for batch_X, batch_y in train_loader:
        batch_X=batch_X.to(device)
        batch_y=batch_y.to(device)
        outputs = model(batch_X).flatten()
        predictions.extend(outputs.squeeze().tolist())
        actuals.extend(batch_y.squeeze().tolist())
    

In [ ]:
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)
print(f"\nTest MSE: {mse:.4e}")
print(f"Test MAE: {mae:.4e}")
print(f"Test R$^2$: {r2:.4f}")
plt.scatter(actuals, predictions)
plt.plot(actuals,actuals)
plt.hlines(np.median(y_train),np.min(y_train),np.max(y_train),color='r')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)

In [ ]:
# model =experimental(featdim)
# if 'lustre' in os.getcwd():
#     model.load_state_dict(torch.load('/lustre/isaac/proj/UTK0022/GMJ/DDGNN/learningcurves/learningcurves/PairNetGCFF/save_model/PairNet_GC_0.75_0.25_cc-pdvz_GDBmix.pth'))
# else:
#     model.load_state_dict(torch.load('/home/grierjones/DDGNN/BigModel/PairNetGCFF/save_model/PairNet_GC_0.80_0.20_cc-pdvz_GDBmix.pth'))

#     # model.load_state_dict(torch.load('/home/grierjones/DDGNN/learningcurves/learningcurves/PairNetGCFF/save_model/PairNet_GC_0.75_0.25_cc-pdvz_GDBmix.pth'))

# num_feats=model.layers[-1].in_features
# model.layers[-1]=torch.nn.Linear(num_feats,1,bias=False)